## Shape of an attractor

The [*Simplest Dissipative Chaotic Flow*](https://sprott.physics.wisc.edu/pubs/paper227.pdf) is a chaotic system, defined by a 3rd order differential equation
\begin{equation}
\dddot{x} = - a\ddot{z} - \dot{z} + b - e^z, \qquad \text{for } a=2.017,\, b=1.
\end{equation}

Its attractor resembles a Mobius strip.

![RotatingAttractor](https://sprott.physics.wisc.edu/fractals/animated/SIMJERKX.GIF)


In this tutorial, we will focus on recovering the homology of that attractor from a discrete sample path. Persistent homology of standard filtration, like the Cech or Vietoris-Rips, are stable in the Hausdorff distance, what yields guarantees for dense-enough i.i.d samples.

In the article that *the attractor has some sheer*, what manifests itself as multiple, discernable trajectories near the lobe of the attractor. It results in regions, where the point-cloud is sparser.

**Question**: Can we modify the construction and filtration of a simplicial complex, so that we see the homology of a strip?

## 1.1 Data

We simulate the dynamic using the `teaspoon.MakeData` module from `teaspoon`.

In [ ]:
import teaspoon.MakeData.DynSysLib.DynSysLib as DSL

t, ts = DSL.DynamicSystems('simplest_quadratic_chaotic_flow', 'chaotic')
print(f"Total time: {t[-1]-t[0]:.3f}\nN points: {t.shape[0]}\nSampling rate: {(t.shape[0]-1)/(t[-1]-t[0]):.3f}")

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("", ""), specs=[[{"type": "xy"}, {"type": "scene"}]])

fig.add_trace(
    go.Scatter(x=t, y=ts[0], mode="lines", name=""),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter3d(x=ts[0], y=ts[1], z=ts[2], mode="lines", name="",),
    row=1, col=2,
)

fig.update_layout(
    showlegend=False,
    xaxis_title="$t$",
    yaxis_title="$x$",
)

fig.show()

We see that with a short time series, the sheer is even worse. This is likely to make more 1-cycles appear.

## 2.2 Persistent homology from a point-cloud

We would like to infer the existence of the cycle using persistent homology. We are in a low-dimensional setting (dim=3), so the [$\alpha$-complex](https://gudhi.inria.fr/python/latest/alpha_complex_ref.html) is the most appropriate tool: it is homotopy-equivalent to the nerve of the collection of thickenings (via the [Cech complex](https://en.wikipedia.org/wiki/%C4%8Cech_complex)).

*Note:* we could also use a Vietoris-Rips complex. However, it has worse approximation guarantees. In addition, since our dataset has 5000 points, the obtained complex is much larger, leading to slower calculations.

Most of the complexes in Gudhi, aside from `CubicalComplex`, rely on the `SimplexTree` datastructure. It is no different for [`AlphaComplex`](https://gudhi.inria.fr/python/latest/alpha_complex_user.html), which just provides a constructor for a particular [`SimplexTree`](https://gudhi.inria.fr/python/latest/simplex_tree_ref.html).

In [ ]:
import numpy as np
from gudhi import AlphaComplex

X = np.stack(ts, axis=1)

st = AlphaComplex(X).create_simplex_tree()
dgm_alpha = st.persistence()

In [ ]:
from utils_gudhi import plot_diagram_gudhi
plot_diagram_gudhi(dgm_alpha)

We see a single, persistent generator in $H_1$. However, zooming-in on the points near the origin, we see several non--trivial cycles, which appear and die well after $H_0$ has stabilised.

Looking at the point-cloud generated by the trajectory, we hypothesize that it is due to the sheer.

In [ ]:
from utils_gudhi import plot_betti_curves_gudhi
fig = plot_betti_curves_gudhi(dgm_alpha)
fig.update_yaxes(range=[-0.2, 15])
fig.update_xaxes(range=[-0.05, 2.])

## 2.3 Distance-to-measure filtrations
If we see the sheer as (unidirectional) noise, we can use the distance-to-measure (DTM) techniques to handle this noisy region (Boissonnat et al., 2018). The DTM filtration (Anai et al., 2020) is available in GUDHI, and also relies on the SimplexTree.

Anai, H. et al. DTM-based Filtrations. arXiv:1811.04757 (2020).\
Boissonnat, J.-D., Chazal, F. & Yvinec, M. Geometric and Topological Inference. (Cambridge University Press, 2018).

In [ ]:
from gudhi.dtm_rips_complex import DTMRipsComplex
dtm = DTMRipsComplex(points=X, k=3, q=1, max_filtration=1.5)
#st = dtm.create_simplex_tree(2) # <- this yields a huge complex (30GB), for max_filtration=0.5, it goes down to 4GB
#dgm_rips_dtm = st.persistence() # <- this is long to compute
#plot_diagram_gudhi(dgm_rips_dtm)

The Vietoris-Rips complex is too big to handle and the homology is long to compute.

**Can we weight the $\alpha$-complex by the DTM?**

A heuristic has been proposed in [`velour.persistence`](https://github.com/raphaeltinarrage/velour/blob/e5e9d085e8407884c7ca7bd96ebf209351c8feff/velour/persistence.py#L776).

```
def AlphaDTMFiltration(X, m, p, dimension_max =2, filtration_max = np.inf):
    '''
    /!\ this is a heuristic method, that speeds-up the computation.
    It computes the DTM-filtration seen as a subset of the Delaunay filtration.
    
    Input:
        X (np.array): size Nxn, representing N points in R^n.
        m (float): parameter of the DTM, in [0,1). 
        p (float): parameter of the DTM-filtration, in [0, +inf) or np.inf.
        dimension_max (int, optional): maximal dimension to expand the complex.
        filtration_max (float, optional): maximal filtration value of the filtration.
    
    Output:
        st (gudhi.SimplexTree): the alpha-DTM filtration.
    '''
    N_tot = X.shape[0]     
    alpha_complex = AlphaComplex(points=X)
    st_alpha = alpha_complex.create_simplex_tree()    
    Y = np.array([alpha_complex.get_point(i) for i in range(N_tot)])
    distances = euclidean_distances(Y)             #computes the pairwise distances
    DTM_values = DTM(X,Y,m)                        #/!\ in 3D, gudhi.AlphaComplex may change the ordering of the points
    
    st = gudhi.SimplexTree()                       #creates an empty simplex tree
    for simplex in st_alpha.get_skeleton(2):       #adds vertices with corresponding filtration value
        if len(simplex[0])==1:
            i = simplex[0][0]
            st.insert([i], filtration  = DTM_values[i])
        if len(simplex[0])==2:                     #adds edges with corresponding filtration value
            i = simplex[0][0]
            j = simplex[0][1]
            value = WeightedRipsFiltrationValue(p, DTM_values[i], DTM_values[j], distances[i][j])
            st.insert([i,j], filtration  = value)
    st.expansion(dimension_max)                    #expands the complex
    return st
```
In particular, it is possible to create a simplicial complex whose 1-skeleton coincides with that of the $\alpha$-complex.

In [ ]:
from velour import AlphaDTMFiltration

st = AlphaDTMFiltration(X, m=0.1, p=1, filtration_max=4)
dgm_alpha_dtm = st.persistence()
plot_diagram_gudhi(dgm_alpha_dtm)

The most persistent generator of $H_1$ is less persistent for the DTM filtration, but the other generators in $H_1$ disappear before we capture the correct $H_0$.

In [ ]:
fig = plot_betti_curves_gudhi(dgm_alpha_dtm)
fig.update_yaxes(range=[-0.2, 15])
fig.update_xaxes(range=[-0.05, 3.3])

## 2.4 Perspectives

### (Non-)orientability
Homology does not see non-orientability. In some cases, it might be useful to resort to richer descriptors, like Stieffel-Whitney classes (Tinarrage, 2020) or Steenrod-persistence (Lupo et al, 2022). The former have been implemented using data structures from GUDHI and are available in the `velour` package.


Lupo, U., Medina-Mardones, A. M. & Tauzin, G. Persistence Steenrod modules. J Appl. and Comput. Topology (2022) doi:10.1007/s41468-022-00093-7. \
Tinarrage, R. Computing persistent Stiefel-Whitney classes of line bundles. (2020).


### Weighted Rips filtration in `giotto-tda`
If we absolutely wanted to calcualte Vietoris-Rips persistence using the DTM distance, we could use `WeightedRipsPersistence` from giotto-tda. It relies on the persistence algorithm from `giotto-ph`, which is currently state-of-the-art.
```
from gtda.homology import WeightedRipsPersistence
Wrp = WeightedRipsPersistence(weights="DTM", homology_dimensions=[0, 1], max_edge_weight=4)
dgm_rips_dtm = Wrp.fit_transform([X])[0]

from gtda.plotting import plot_diagram
plot_diagram(dgm_rips_dtm)
```
*It is not released in its current version.*